In [6]:
import pandas as pd
import numpy as np
import plotly.express as px

final_cost_report = pd.read_csv("final_cost_report2.csv")


In [7]:
# Reemplazar strings "nan" por NaN reales
final_cost_report.replace("nan", np.nan, inplace=True)

# Filtrar solo cuentas generales
cuentas_generales = final_cost_report[final_cost_report["REF"] == "Cuenta General"].copy()

# Agrupar por TIPO
resumen_tipo = cuentas_generales.groupby("TIPO", as_index=False)[["COST TO DATE", "BUDGET"]].sum()

# Calcular columnas adicionales
resumen_tipo["TOTAL CTD"] = resumen_tipo["COST TO DATE"]
resumen_tipo["EFC"] = resumen_tipo["COST TO DATE"]  # Estimate Final Cost
resumen_tipo["VARIANCE"] = resumen_tipo["BUDGET"] - resumen_tipo["COST TO DATE"]

# Reordenar columnas
resumen_tipo = resumen_tipo[["TIPO", "COST TO DATE", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE"]]

# Agregar fila TOTAL
fila_total = resumen_tipo[["COST TO DATE", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE"]].sum()
fila_total["TIPO"] = "TOTAL"
resumen_tipo = pd.concat([resumen_tipo, pd.DataFrame([fila_total])], ignore_index=True)

# Formatear con separadores de miles
def formatear(val):
    return f"${val:,.0f}"

for col in ["COST TO DATE", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE"]:
    resumen_tipo[col] = resumen_tipo[col].apply(lambda x: formatear(x))

# Mostrar tabla en consola
print("\n" + "-"*90)
print("{:<18} {:>15} {:>15} {:>15} {:>15} {:>15}".format(
    "Concept", "COST TO DATE", "TOTAL CTD", "EFC", "BUDGET", "VARIANCE $"
))
print("-"*90)
for _, row in resumen_tipo.iterrows():
    print("{:<18} {:>15} {:>15} {:>15} {:>15} {:>15}".format(
        row["TIPO"], row["COST TO DATE"], row["TOTAL CTD"],
        row["EFC"], row["BUDGET"], row["VARIANCE"]
    ))
print("-"*90)

def formatear(val):
    return f"${val:,.0f}"

def imprimir_tabla(df, titulo):
    # Definir los anchos de columna
    widths = [8, 15, 15, 15]
    headers = ["ACCT", "COST TO DATE", "BUDGET", "DIFFERENCE"]

    # Borde superior
    print("\n" + "═"*60)
    print(f"{titulo}".center(60))
    print("═"*60)
    # Encabezados
    header_row = "║ {:^8} ║ {:^15} ║ {:^15} ║ {:^15} ║".format(*headers)
    print(header_row)
    print("╠" + "════════╬" + "═══════════════╬"*2 + "═══════════════╣")
    # Filas
    for _, row in df.iterrows():
        print("║ {:>8} ║ {:>15} ║ {:>15} ║ {:>15} ║".format(
            row['ACCT'], row['COST TO DATE'], row['BUDGET'], row['VARIANCE']
        ))
    # Borde inferior
    print("═"*60)

# Beneficiadas (Top 15)
beneficiadas = cuentas_generales.sort_values("VARIANCE", ascending=False).head(15).copy()
beneficiadas["COST TO DATE"] = beneficiadas["COST TO DATE"].apply(formatear)
beneficiadas["BUDGET"] = beneficiadas["BUDGET"].apply(formatear)
beneficiadas["VARIANCE"] = beneficiadas["VARIANCE"].apply(formatear)

imprimir_tabla(beneficiadas, "Cuentas Más Beneficiadas")

# Impactadas (Bottom 15)
impactadas = cuentas_generales.sort_values("VARIANCE", ascending=True).head(15).copy()
impactadas["COST TO DATE"] = impactadas["COST TO DATE"].apply(formatear)
impactadas["BUDGET"] = impactadas["BUDGET"].apply(formatear)
impactadas["VARIANCE"] = impactadas["VARIANCE"].apply(formatear)

imprimir_tabla(impactadas, "Cuentas Más Impactadas")






------------------------------------------------------------------------------------------
Concept               COST TO DATE       TOTAL CTD             EFC          BUDGET      VARIANCE $
------------------------------------------------------------------------------------------
Above the Line         $37,179,095     $37,179,095     $37,179,095     $35,701,887     $-1,477,207
Below the Line         $68,434,405     $68,434,405     $68,434,405     $69,999,853      $1,565,448
Others                 $14,267,410     $14,267,410     $14,267,410     $14,989,184        $721,774
Post                   $14,491,072     $14,491,072     $14,491,072     $14,406,986        $-84,086
TOTAL                 $134,371,983    $134,371,983    $134,371,983    $135,097,911        $725,928
------------------------------------------------------------------------------------------

════════════════════════════════════════════════════════════
                  Cuentas Más Beneficiadas                  
═════════

In [14]:

# Filtrar solo cuentas generales si quieres
df = final_cost_report[final_cost_report["REF"] == "Cuenta General"].copy()

# Agrupar por TIPO y sumar COST TO DATE
resumen = df.groupby("TIPO", as_index=False)["COST TO DATE"].sum()
resumen = resumen.sort_values("COST TO DATE", ascending=False)

# Gráfico de rosca (donut)
fig = px.pie(
    resumen,
    values="COST TO DATE",
    names="TIPO",
    title="<b>Porcentaje de uso por TIPO </b>",
    hole=0.5,  # Hace la rosca
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    pull=[0.02]*len(resumen),  # Separa un poquito cada porción
    marker=dict(line=dict(color='white', width=2))
)

fig.update_layout(
    showlegend=True,
    title_font_size=22,
    legend_title_text='Tipo de Cuenta',
    legend=dict(font=dict(size=14))
)

fig.show()

La gráfica muestra la distribución porcentual del uso del presupuesto real (Cost to Date) por tipo de cuenta general. Cada segmento representa un tipo de gasto dentro del proyecto, permitiendo visualizar de forma clara cuáles categorías concentran la mayor parte del presupuesto ejecutado hasta la fecha. Esta visualización facilita identificar rápidamente los rubros que absorben más recursos y analizar si la distribución es acorde a los objetivos y prioridades del proyecto. El gráfico es útil para detectar posibles desviaciones, oportunidades de optimización y áreas donde se puede implementar una gestión financiera más eficiente